## 딥러닝 6주차

### 과적합 방지기법

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

#### Dropout과 정규화

In [5]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class CNNWithDropoutAndBatchNorm(nn.Module):
    def __init__(self):
        super(CNNWithDropoutAndBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 56 * 56, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNNWithDropoutAndBatchNorm()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

#### 데이터 증강

In [ ]:
# 데이터 증강 적용
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

### 모델 평가와 검증 및 Pytorch 문법 정리

#### 1. 모델 구축 및 학습

In [ ]:
# 기본 모델 구축

import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(10, 20)

    def forward(self, x):
        x = self.layer1(x)
        return x


In [ ]:
# 손실함수

loss_fn = nn.CrossEntropyLoss()  # 분류 문제에 주로 사용
loss_fn = nn.CrossEntropyLoss()  # 회귀 문제에 주로 사용

In [ ]:
# 최적화 알고리즘

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # 확률적 경사 하강법 최적화 알고리즘
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adam 최적화 알고리즘

#### 2. 데이터 로드 및 전처리

In [ ]:
# 사용자 정의 데이터셋을 만들기 위한 기본 클래스

from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [ ]:
# 미니 배치 학습을 위한 데이터 로더

from torch.utils.data import DataLoader

dataset = MyDataset(data, targets)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# 이미지 데이터 변환을 위한 유틸리티

from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


#### 3. GPU 사용

In [ ]:
# 모델을 GPU로 이동

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# 텐서를 GPU로 이동

inputs, targets = inputs.to(device), targets.to(device)

#### 4. 모델 기법별 API

In [ ]:
# CNN

conv_layer = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)

In [ ]:
# RNN

rnn_layer = nn.RNN(input_size=10, hidden_size=20, num_layers=2, batch_first=True)  # vanilla RNN
lstm_layer = nn.LSTM(input_size=10, hidden_size=20, num_layers=2, batch_first=True)  # LSTM
gru_layer = nn.GRU(input_size=10, hidden_size=20, num_layers=2, batch_first=True)  # GRU

In [ ]:
# Transformer

transformer_model = nn.Transformer(nhead=8, num_encoder_layers=6)  # 트랜스포머 모델
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)  # 트랜스포머 인코더 레이어

#### 5. 유틸리티 함수

In [ ]:
# 모델 저장
torch.save(model.state_dict(), 'model.pth')  

# 모델 로드
model.load_state_dict(torch.load('model.pth'))
model.eval()

In [ ]:
# 모델을 학습 모드로 설정
model.train()

# 모델을 평가 모드로 설정
model.eval()